# Portfolio

In [1]:
import pandas as pd
import numpy as np
from beakerx import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# load prices from a csv file
prices=pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [3]:
# plot them using beakerx
SimpleTimePlot(prices, prices.keys())

In [4]:
# construct a 1/n portfolio, assets that come in later will initially not have any weight
def f(x):
    # how many assets are alive?
    n = x.notnull().sum()
    # create a new series (this seems to be slow, there must be a more elegant solution?)
    y = pd.Series(index=x.index)
    if n > 0:
        y[x.notnull()] = 1.0/n
    return y

In [5]:
p = pd.Series({"A": 10.0, "B": 20.0, "C": np.nan}) 
p = pd.DataFrame(p).transpose()
g = p.apply(f, axis=1)
g

,A,B,C
0,0.5,0.5,NaN


In [13]:
from pyutil.portfolio.portfolio import Portfolio
p = Portfolio(prices=prices, weights=prices.ffill().apply(f, axis=1))

In [14]:
print(dir(p))

['_Portfolio__prices', '_Portfolio__series2frame', '_Portfolio__weights', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'apply', 'asset_returns', 'assets', 'cash', 'copy', 'empty', 'fromPosition', 'head', 'index', 'iron_threshold', 'iron_time', 'leverage', 'loop', 'merge', 'nav', 'position', 'prices', 'rename', 'returns', 'sector', 'subportfolio', 'tail', 'to_frame', 'truncate', 'weight_current', 'weighted_returns', 'weights']


## Sector analysis

It is possible to assign each asset to a sector via sectormaps. 


In [15]:
sectormap = {"A": "S1", "B": "S1", "C": "S2", "D": "S2", "E": "S2", "F": "S2", "G": "S3"}
p.tail(5).sector(symbolmap=sectormap)

,S1,S2,S3
2015-04-16,0.285714,0.571429,0.142857
2015-04-17,0.285714,0.571429,0.142857
2015-04-20,0.285714,0.571429,0.142857
2015-04-21,0.285714,0.571429,0.142857
2015-04-22,0.285714,0.571429,0.142857


## Rebalancing

Daily rebalancing is somewhat expensive. It is possible to "iron" the portfolio and rebalance either
* on a fixed grid in time
* with respect to threshold that shall not be exceeded

In [16]:
x = p.iron_time("3M")
1e6*x.position

,A,B,C,D,E,F,G
2013-01-01,85.350012,6.128057,2.283884,38.247002,97.685441,52.690509,56.712072
2013-01-02,85.254544,6.121203,2.281329,38.204221,97.576176,52.631573,56.648638
2013-01-03,85.409885,6.132356,2.285486,38.273832,97.753967,52.727471,56.751856
2013-01-04,85.612939,6.146935,2.290919,38.364825,97.986368,52.852826,56.886778
2013-01-07,85.828507,6.162413,2.296688,38.461425,98.233092,52.985906,57.030015
...,...,...,...,...,...,...,...
2015-04-16,105.014015,5.501023,2.874025,51.837721,67.576200,40.225647,42.754070
2015-04-17,105.890986,5.546962,2.898026,52.270617,68.140527,40.561571,43.111108
2015-04-20,106.102349,5.558034,2.903810,52.374952,68.276539,40.642534,43.197160
2015-04-21,105.666160,5.535185,2.891873,52.159637,67.995853,40.475451,43.019576
